In [1]:
%useLatestDescriptors
%use dataframe

In [2]:
val apiKey = "YOUR API KEY"

In [3]:
fun load(path: String) = DataRow.read("https://www.googleapis.com/youtube/v3/$path&key=$apiKey")

In [4]:
fun load(path: String, maxPages: Int): AnyFrame {
    val rows = mutableListOf<AnyRow>()
    var pagePath = path
    do {
        val row = load(pagePath)
        rows.add(row)
        val next = row.getValueOrNull<String>("nextPageToken")
        pagePath = path + "&pageToken=" + next
    } while(next != null && rows.size < maxPages)
    return rows.concat()
}

In [5]:
val df = load("search?q=cute%20cats&maxResults=50&part=snippet", 5)
df

DataFrame: rowsCount = 5, columnsCount = 7

In [6]:
val items = df.items.concat()
items

In [7]:
val videos = items.dropNulls { id.videoId }
    .select { id.videoId named "id" and snippet }
    .distinct()
videos

In [8]:
val parsed = videos.parse()

In [9]:
val loaded = parsed.convert { dfsOf<URL>() }.with { IMG(it, maxHeight = 150) }
    .add("video") { IFRAME("http://www.youtube.com/embed/$id") }

In [10]:
val clean = loaded.move { snippet.channelId and snippet.channelTitle }.under("channel")
    .move { snippet.title and snippet.publishedAt }.toTop()
    .remove { snippet }
clean

... showing only top 20 of 241 rows DataFrame: rowsCount = 241, columnsCount = 5

In [11]:
val statPages = clean.id.chunked(50).map {
    val ids = it.joinToString("%2C")
    load("videos?part=statistics&id=$ids")
}
statPages

ColumnGroup: name = "id", rowsCount = 5, columnsCount = 4

In [12]:
val stats = statPages.items.concat().select { id and statistics.all() }.parse()
stats

... showing only top 20 of 241 rows DataFrame: rowsCount = 241, columnsCount = 5

In [13]:
val joined = clean.join(stats)
joined

... showing only top 20 of 241 rows DataFrame: rowsCount = 241, columnsCount = 9

In [14]:
val view by column<Int>()

val channels = joined.groupBy { channel }.sortByCount().aggregate {
    viewCount.sum() into view
    
    val last = maxBy { publishedAt }
    last.title into "last title"
    last.publishedAt into "time"
    last.viewCount into "viewCount"
}.sortByDesc(view).flatten()
channels

... showing only top 20 of 113 rows DataFrame: rowsCount = 113, columnsCount = 6

In [15]:
%use lets-plot

In [16]:
letsPlot(channels.sortBy { viewCount.desc() }.toMap()) +
    geomBar(stat = Stat.identity) {
        x = "channelTitle"
        y = "viewCount"
    }

In [17]:
val growth = joined.select { publishedAt and viewCount }
    .sortBy { publishedAt }
    .convert { all() }.toLong()
    .cumSum { viewCount }

In [18]:
letsPlot(growth.toMap()) + 
    geomArea {
        x = "publishedAt"
        y = "viewCount"
    } + scale_x_datetime()